## Compare the results of handcrafted, baselines, and HumemAI


In [5]:
from typing import Literal
import shutil
import os
from glob import glob
from humemai.utils import read_yaml
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from typing import Literal
import pandas as pd


def get_handcrafted(
    size: Literal["xxs", "xs", "s", "m", "l", "xl", "xxl"],
    include_pretrain_semantic: bool = True,
    base_path: str = "training-results/non-equal-object-probs/",
) -> pd.DataFrame:
    """Get hand-crafted results.

    room_size=xxs   num_obs=6.0     max_obs=6   min_obs=6
    room_size=xs    num_obs=6.52    max_obs=8   min_obs=5
    room_size=s     num_obs=5.64    max_obs=7   min_obs=5
    room_size=m     num_obs=6.3     max_obs=10  min_obs=5
    room_size=l     num_obs=5.32    max_obs=8   min_obs=5
    room_size=xl    num_obs=5.58    max_obs=7   min_obs=5
    room_size=xxl   num_obs=6.0     max_obs=8   min_obs=5

    Args:
        size: room size

    """

    df = pd.read_csv(
        os.path.join(
            base_path, f"handcrafted/hand-crafted-results-room_size={size}.csv"
        )
    )

    if not include_pretrain_semantic:
        df = df[
            (df["pretrain_semantic"] == False)
            | (df["pretrain_semantic"].str.lower() == "false")
        ]

    df["test_mean"] = pd.to_numeric(df["test_mean"], errors="coerce")
    df_sorted = df.sort_values(
        by=["long_capacity", "test_mean"], ascending=[True, False]
    )
    df = df_sorted.groupby("long_capacity").first().reset_index()

    # add column "room_size":
    df["room_size"] = size

    # Rename columns
    df.rename(
        columns={
            "long_capacity": "capacity",
            "test_mean": "test",
            "test_std": "std_test",
        },
        inplace=True,
    )

    df["val"] = np.nan
    df["agent_type"] = "handcrafted"
    df["#_runs"] = 5
    df["terminates_at"] = 99

    # Reorder columns in df1
    df = df[
        [
            "test",
            "std_test",
            "val",
            "#_runs",
            "capacity",
            "agent_type",
            "pretrain_semantic",
            "semantic_decay_factor",
            "room_size",
            "explore_policy",
            "qa_function",
            "mm_policy",
        ]
    ]

    # rename columns
    df.rename(columns={"mm_policy": "mm"}, inplace=True)
    df.rename(columns={"qa_function": "qa"}, inplace=True)
    df.rename(columns={"explore_policy": "explore"}, inplace=True)

    return df


def determine_hyper_parameters(train: dict) -> dict:
    """Determine hyper parameters."""
    hyper_parameters = {}

    hyper_parameters["capacity"] = train["capacity"]["long"]
    hyper_parameters["pretrain_semantic"] = train["pretrain_semantic"]
    hyper_parameters["semantic_decay_factor"] = train["semantic_decay_factor"]
    hyper_parameters["room_size"] = train["env_config"]["room_size"]
    hyper_parameters["#_iter"] = train["num_iterations"]
    hyper_parameters["replay_size"] = train["replay_buffer_size"]
    # hyper_parameters["warm_start"] = train["warm_start"]
    hyper_parameters["terminates_at"] = train["env_config"]["terminates_at"]
    hyper_parameters["target_update"] = train["target_update_interval"]
    if isinstance(train["gamma"], dict):
        hyper_parameters["gamma_mm"] = train["gamma"]["mm"]
        hyper_parameters["gamma_explore"] = train["gamma"]["explore"]
    else:
        hyper_parameters["gamma_mm"] = train["gamma"]
        hyper_parameters["gamma_explore"] = train["gamma"]

    hyper_parameters["agent_type"] = "dqn"
    hyper_parameters["relu_between"] = train["dqn_params"]["relu_between_gcn_layers"]
    hyper_parameters["dropout_between"] = train["dqn_params"][
        "dropout_between_gcn_layers"
    ]
    hyper_parameters["num_layers"] = train["dqn_params"]["gcn_layer_params"][
        "num_layers"
    ]
    hyper_parameters["batch_size"] = train["batch_size"]
    hyper_parameters["emb_dim"] = train["dqn_params"]["gcn_layer_params"][
        "embedding_dim"
    ]
    hyper_parameters["triple_weight"] = train["dqn_params"]["gcn_layer_params"][
        "triple_qual_weight"
    ]
    hyper_parameters["intrinsic"] = train["intrinsic_explore_reward"]
    hyper_parameters["mm"] = train["mm_policy"]
    hyper_parameters["explore"] = train["explore_policy"]
    hyper_parameters["gcn"] = train["dqn_params"]["gcn_layer_params"]["type"]
    hyper_parameters["scale"] = train["scale_reward"]
    hyper_parameters["qa_function"] = train["qa_function"]

    if "learning_rate" in train:
        hyper_parameters["lr"] = train["learning_rate"]

    return hyper_parameters


def nanmean(data):
    return None if np.isnan(data).any() else round(np.mean(data))


def nanstd(data):
    return None if np.isnan(data).any() else round(np.std(data))


def nanmax(data):
    return None if np.isnan(data).any() else round(np.max(data))


def nanmin(data):
    return None if np.isnan(data).any() else round(np.min(data))


def get_dataframe(
    room_size: Literal["xxs", "xs", "s", "m", "l", "xl", "xxl"],
    base_path: str = "training-results/non-equal-object-probs/",
) -> pd.DataFrame:
    paths = glob(os.path.join(base_path, f"dqn/room_size={room_size}/*/*/results.yaml"))

    if len(paths) == 0:
        return pd.DataFrame()

    final = {}
    for result_path in tqdm(paths):
        result = read_yaml(result_path)
        train_path = result_path.replace("results.yaml", "train.yaml")
        train = read_yaml(train_path)

        val_score = max([foo["mean"] for foo in result["validation_score"]])
        test_score = result["test_score"]["mean"]

        hp = determine_hyper_parameters(train)

        hp_str = str(hp)
        if hp_str in final:
            final[hp_str]["val"].append(val_score)
            final[hp_str]["test"].append(test_score)
            final[hp_str]["path"].append(result_path.split("/")[5].split(".")[-1])

        else:
            final[hp_str] = {
                "val": [val_score],
                "test": [test_score],
                "hyper_parameters": hp,
                "path": [result_path.split("/")[5].split(".")[-1]],
            }

    df_list = []
    for key in final:
        data = final[key]
        hp = data["hyper_parameters"]
        df_list.append(
            {
                "test": nanmean(data["test"]),
                "std_test": nanstd(data["test"]),
                # "test_max": nanmax(data["test"]),
                # "test_min": nanmin(data["test"]),
                "val": nanmean(data["val"]),
                # "std_val": nanstd(data["val"]),
                "#_runs": len(data["test"]),
                "capacity": hp.get("capacity", None),
                "agent_type": hp.get("agent_type", None),
                "pretrain_semantic": hp.get("pretrain_semantic", None),
                "semantic_decay_factor": hp.get("semantic_decay_factor", None),
                "room_size": hp.get("room_size", None),
                "mm": hp.get("mm", None),
                "qa": hp.get("qa_function", None),
                "explore": hp.get("explore", None),
                "#_iter": hp.get("#_iter", None),
                "replay_size": hp.get("replay_size", None),
                # "warm_start": hp.get("warm_start", None),
                "terminates_at": hp.get("terminates_at", None),
                "target_update": hp.get("target_update", None),
                "gamma_mm": hp.get("gamma_mm", None),
                "gamma_explore": hp.get("gamma_explore", None),
                "relu_between": hp.get("relu_between", None),
                "dropout_between": hp.get("dropout_between", None),
                "num_layers": hp.get("num_layers", None),
                "batch_size": hp.get("batch_size", None),
                "emb_dim": hp.get("emb_dim", None),
                "triple_weight": hp.get("triple_weight", None),
                "intrinsic": hp.get("intrinsic", None),
                "lr": hp.get("lr", None),
                "gcn": hp.get("gcn", None),
                "scale": hp.get("scale", None),
                "path": data["path"],
            }
        )

    df = pd.DataFrame(df_list)
    df_sorted = df.sort_values(
        by=["capacity", "test"],
        ascending=[True, False],
    )
    return df_sorted


# Function to add blank rows and flag them
def add_blank_rows_and_flag(df):
    # Create a list to hold the new rows
    new_rows = []
    previous_capacity = None

    # Iterate through the dataframe rows
    for index, row in df.iterrows():
        if previous_capacity is not None and row["capacity"] != previous_capacity:
            # Add a blank row and flag it when the capacity changes
            blank_row = pd.Series({col: "" for col in df.columns})
            blank_row["flag"] = True
            new_rows.append(blank_row)
        # Append the current row
        new_row = row.copy()
        new_row["flag"] = False
        new_rows.append(new_row)
        previous_capacity = row["capacity"]

    # Create a new dataframe from the new rows
    new_df = pd.DataFrame(new_rows).reset_index(drop=True)
    return new_df


# Function to highlight the flagged rows
def highlight_blank_rows(row):
    if row.flag:
        return ["background-color: yellow"] * len(row)
    else:
        return [""] * len(row)


def get_all_data(
    size: Literal["xxs", "xs", "s", "m", "l", "xl", "xxl"],
    include_pretrain_semantic: bool = True,
    base_path: str = "training-results/non-equal-object-probs/",
    include_handcrafted: bool = True,
) -> pd.DataFrame:
    """Get all data."""

    if include_handcrafted:
        df_1 = get_handcrafted(
            size,
            include_pretrain_semantic=include_pretrain_semantic,
            base_path=base_path,
        )
        df_2 = get_dataframe(size, base_path=base_path)
        df = pd.concat([df_1, df_2], ignore_index=True)

        df = df.sort_values(
            by=["capacity", "test"],
            ascending=[True, False],
        )
    else:
        df = get_dataframe(size, base_path=base_path)

    df.rename(columns={"semantic_decay_factor": "sem_decay"}, inplace=True)
    df.rename(columns={"pretrain_semantic": "pretrain_sem"}, inplace=True)
    df.rename(columns={"history_block_size": "history"}, inplace=True)

    # Add blank rows and flag them in the dataframe
    df_with_blanks = add_blank_rows_and_flag(df)

    # Apply the highlight function
    df_with_blanks_styled = df_with_blanks.style.apply(highlight_blank_rows, axis=1)

    # Apply number formatting to styled DataFrame
    df_with_blanks_styled = df_with_blanks_styled.format(na_rep="NaN", precision=4)

    return df, df_with_blanks_styled


def filter_paths(
    room_size: Literal["xxs", "xs", "s", "m", "l", "xl", "xxl"],
    agent_type: Literal["baseline", "episodic", "semantic", "hybrid"],
    num_iterations: int,
    capacity: int,
    semantic_decay_factor: float,
    base_path: str,
) -> list:
    paths = glob(
        os.path.join(base_path, f"baselines/room_size={room_size}/*/*/results.yaml")
    ) + glob(
        os.path.join(base_path, f"dqn/room_size={room_size}/*/*/explore/results.yaml")
    )

    if len(paths) == 0:
        return []

    filtered_paths = []
    for path in tqdm(paths):
        if "baseline" in path:
            train_path = path.replace("results.yaml", "train.yaml")
        else:
            train_path = path.replace("explore/results.yaml", "train.yaml")

        train_data = read_yaml(train_path)

        # Determine hyperparameters to check agent type and capacity
        hp = determine_hyper_parameters(train_data)

        if hp.get("#_iter") != num_iterations:
            continue

        if hp.get("semantic_decay_factor") != semantic_decay_factor:
            continue

        # Check the agent type
        if hp.get("agent_type") != agent_type:
            continue

        # Check the capacity range
        if hp.get("capacity") != capacity:
            continue

        # If all conditions are met, add the path to the filtered list
        filtered_paths.append(path)

    return filtered_paths

In [7]:
# filter_paths(
#     room_size="xl-different-prob",
#     num_iterations=10000,
#     agent_type="hybrid",
#     capacity=12,
#     semantic_decay_factor=0.99,
#     base_path="training-results/non-equal-object-probs/",
# )

# for foo in filter_paths_by_num_iterations("xl-different-prob"):
#     dir_path = '/'.join(foo.split('/')[:-2])
#     shutil.rmtree(dir_path)

# # save dataframe as markdown
# df.to_markdown("./hp-tuning/xl-different-prob.md", index=False)

df, df_styled = get_all_data(
    "xxl-different-prob",
    include_pretrain_semantic=True,
    base_path="training-results/non-equal-object-probs/",
)
df_styled

100%|██████████| 25/25 [00:26<00:00,  1.07s/it]


,test,std_test,val,#_runs,capacity,agent_type,pretrain_sem,sem_decay,room_size,explore,qa,mm,#_iter,replay_size,terminates_at,target_update,gamma_mm,gamma_explore,relu_between,dropout_between,num_layers,batch_size,emb_dim,triple_weight,intrinsic,lr,gcn,scale,path,flag
0,23.9400,1.4854,NaN,5,2,handcrafted,exclude_walls,0.9900,xxl-different-prob,avoid_walls,latest_strongest,handcrafted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,True
2,33.2400,2.7645,NaN,5,6,handcrafted,include_walls,0.8000,xxl-different-prob,avoid_walls,latest_strongest,handcrafted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,True
4,46.8400,6.8066,NaN,5,12,handcrafted,False,0.9900,xxl-different-prob,avoid_walls,latest_strongest,handcrafted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,True
6,78.4400,6.3140,NaN,5,24,handcrafted,False,0.9900,xxl-different-prob,avoid_walls,latest_strongest,handcrafted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
7,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,True
8,156.0000,0.0000,167.0000,1,48,dqn,False,0.8000,xxl-different-prob,rl,latest_strongest,rl,10000.0000,1000.0000,99.0000,50.0000,0.9000,0.9000,True,False,2.0000,32.0000,64.0000,0.8000,0.0000,0.0010,stare,False,['909809'],False
9,123.0000,0.0000,115.0000,1,48,dqn,False,0.8000,xxl-different-prob,rl,latest_strongest,rl,10000.0000,10000.0000,99.0000,10.0000,0.9000,0.9000,True,False,2.0000,32.0000,64.0000,0.8000,0.0000,0.0010,stare,False,['536966'],False
